In [1]:
! pip install -U accelerate
# ! pip install -U transformers

In [2]:
!pip install transformers==4.30

In [3]:
import transformers
import accelerate
transformers.__version__, accelerate.__version__

('4.30.0', '0.24.0')

In [ ]:
!pip install datasets

In [ ]:
!pip install sentencepiece

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [32]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [46]:
data = pd.read_csv('val_context.csv')
qna = pd.read_csv('val_qna.csv')

df = pd.merge(data, qna, left_on='index', right_on='context', how='inner')
df.head()

,index,context_x,context_y,question,answer
0,0,Hypertensive heart disease is the No. 1 cause ...,0,What are the symptoms of heart failure?,Symptoms of heart failure include: Shortness o...
1,0,Hypertensive heart disease is the No. 1 cause ...,0,Can high blood pressure bring on heart failure?,High blood pressure may also bring on heart fa...
2,0,Hypertensive heart disease is the No. 1 cause ...,0,What tests are used to help diagnose hypertens...,Your doctor will look for certain signs of hyp...
3,0,Hypertensive heart disease is the No. 1 cause ...,0,What is hypertensive heart disease?,It refers to a group of disorders that include...
4,0,Hypertensive heart disease is the No. 1 cause ...,0,What are the symptoms of ischemic heart disease?,Symptoms of ischemic heart disease may include...


In [47]:
context_question = []
for index, row in df.iterrows():
    prompt = 'Context: ' + row['context_x'] + '\n\nQuestion: ' + row['question'] + '\n\nAnswer:'
    # if(len(prompt) > 356):
    #   prompt = 'Context: ' + row['context_x'][:256] + '\n\nQuestion: ' + row['question'] + '\n\nAnswer:'
    context_question.append(prompt)

In [48]:
context_question[0]

"Context: Hypertensive heart disease is the No. 1 cause of death associated with high blood pressure. It refers to a group of disorders that includes heart failure, ischemic heart disease, and left ventricular hypertrophy (excessive thickening of the heart muscle). Heart failure does not mean the heart has stopped working. Rather, it means that the heart's pumping power is weaker than normal or the heart has become less elastic. With heart failure, blood moves through the heart's pumping chambers less effectively, and pressure in the heart increases, making it harder for your heart to deliver oxygen and nutrients to your body. To compensate for reduced pumping power, the heart's chambers respond by stretching to hold more blood. This keeps the blood moving, but over time, the heart muscle walls may weaken and become unable to pump as strongly. As a result, the kidneys often respond by causing the body to retain fluid (water) and sodium. The resulting fluid buildup in the arms, legs, an

In [49]:
Answer = df['answer'].tolist()

In [50]:
import torch
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd

# Define the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-large')
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')

# Define the training data
inputs = context_question
outputs = Answer
data = []
for i in range(len(inputs)):
  data.append([inputs[i], outputs[i]])

df = pd.DataFrame(data, columns=["input", "output"])
ds = DatasetDict()
ds['train'] = Dataset.from_pandas(df)
train_data = ds['train']

# train_data = Dataset.from_dict({'input': inputs, 'output': outputs})

# Define the fine-tuning function
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained('t5-small')

In [ ]:
def tokenise_ds(example):
    return tokenizer.prepare_seq2seq_batch(src_texts=example["input"], tgt_texts=example["output"])

tokenized_datasets = train_data.map(tokenise_ds, batched=True)

In [52]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model=model)

In [53]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    # evaluation_strategy='steps',
    # eval_steps=10,
    # save_steps=10,
    learning_rate=3e-4,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    # weight_decay=0.01,
    # push_to_hub=False,
    # logging_dir='./logs',
    # logging_steps=10,
    # load_best_model_at_end=True,
    # metric_for_best_model='eval_loss',
    # greater_is_better=False
)

# Define the training function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {'eval_loss': torch.nn.functional.cross_entropy(predictions, labels).item()}

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    #lambda data: {'input_ids': tokenizer(data['input'], padding=True, truncation=True, max_length=512, return_tensors='pt').input_ids, 'labels': tokenizer(data['output'], padding=True, truncation=True, max_length=512, return_tensors='pt').input_ids},
    # compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# Start the training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.620400
1000,0.398500
1500,0.423900
2000,0.368700
2500,0.397000
3000,0.359200
3500,0.385600


TrainOutput(global_step=3587, training_loss=0.42129356229009485, metrics={'train_runtime': 404.3478, 'train_samples_per_second': 8.871, 'train_steps_per_second': 8.871, 'total_flos': 485471041880064.0, 'train_loss': 0.42129356229009485, 'epoch': 1.0})

In [54]:
input_ids = tokenizer("what are the symptoms of heart disease?", return_tensors="pt").input_ids  # Batch size 1

outputs = model.generate(input_ids, max_length=512)
print(tokenizer.decode(outputs[0]))

<pad> a sluggish heartbeat</s>


In [56]:
input_ids = tokenizer("what is diabetes?", return_tensors="pt").input_ids  # Batch size 1

outputs = model.generate(input_ids, max_length=512)
print(tokenizer.decode(outputs[0]))

<pad> Diabetes is a disease characterized by the presence of high levels of glucose in the blood.</s>


In [ ]:
inputs = tokenizer(context_question,
                   return_tensors="pt",
                   padding='max_length',
                  #  truncation=True,
                    max_length=2500,
                  ).to('cuda')

# Answer = df['answer'][:100].tolist()
Answer = df['answer'][:100]

outputs = tokenizer(Answer,
                   return_tensors="pt",
                   padding='max_length',
                  #  truncation=True,
                    max_length=2500,
                  ).to('cuda')


# train_dataset = TensorDataset(torch.tensor(inputs.input_ids), torch.tensor(outputs.input_ids))

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {key: self.inputs[key][idx] for key in self.inputs}

In [ ]:
dataset = CustomDataset(inputs)
batch_size = 1  # Adjust the batch size as needed
dataloader = DataLoader(dataset, batch_size=batch_size)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for batch in dataloader:
    batch = {key: batch[key].to('cuda') for key in batch}
    with torch.no_grad():
        outputs = model.generate(**batch).to('cuda')
#     answers.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
        torch.cuda.empty_cache()  # Clear GPU cache

    print(tokenizer.batch_decode(outputs, skip_special_tokens=True, max_length=200))
# outputs = model.generate(**inputs).to('cuda')
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))